In [113]:
from models.SimpleCNN import SimpleCNN
from models.AlexNet import AlexNet
from models.ResNet import ResNet, ResidualBlock
import torch
import random
import numpy as np
import sys
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [99]:
an = AlexNet()

In [108]:
(3*11*11 + 11 + 64*5*5 + 64 + 192*3*3 + 192 + 384*3*3 + 384 \
 + 256*3*3 + 9216*4096 + 4096 + 4096*4096 + 4096 + 4096*1000 + 1000)*4/1024/1024

223.69860076904297

In [109]:
(9216*4096 + 4096*4096 + 4096*1000)*4/1024/1024

223.625

In [46]:
net = SimpleCNN()

In [61]:
net.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.0333,  0.0651, -0.0805, -0.0726, -0.0010],
                        [-0.0628, -0.1703,  0.1084, -0.0266,  0.0225],
                        [-0.0297,  0.0236,  0.1508,  0.0582,  0.0581],
                        [ 0.1026,  0.1341,  0.0807,  0.1285,  0.1105],
                        [-0.1772,  0.1706,  0.1381,  0.1293,  0.0070]]],
              
              
                      [[[-0.0370,  0.1529,  0.0437,  0.0769, -0.0402],
                        [-0.1830, -0.1313,  0.1884, -0.0552,  0.0977],
                        [-0.1966,  0.0685, -0.1407, -0.1272, -0.0549],
                        [-0.0218,  0.1325, -0.1874,  0.0541, -0.1716],
                        [-0.1121, -0.0900, -0.1455, -0.0328, -0.0427]]],
              
              
                      [[[-0.1476,  0.0029, -0.1518,  0.0579, -0.0667],
                        [ 0.0224,  0.1465,  0.1510, -0.1387, -0.0958],
                        [ 0.0939, -0.0736,  0.1566, -0

In [60]:
for p in net.parameters():
    print(p.shape)

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [95]:
perBytes = 4
(6*1*5*5+16*6*5*5+120*400+84*120+10*84+6+16+120+84+10)*perBytes/1024

241.0390625

In [102]:
torch.save(an.state_dict(), "./net.pth")

In [93]:
for p in net.parameters():
    print(p[0][0][0][0])
    print(p[0][0][0][0].dtype)
    break

tensor(-0.0333, grad_fn=<SelectBackward0>)
torch.float32


In [48]:
for p in net.parameters():
    print(p)
    break

Parameter containing:
tensor([[[[-0.0333,  0.0651, -0.0805, -0.0726, -0.0010],
          [-0.0628, -0.1703,  0.1084, -0.0266,  0.0225],
          [-0.0297,  0.0236,  0.1508,  0.0582,  0.0581],
          [ 0.1026,  0.1341,  0.0807,  0.1285,  0.1105],
          [-0.1772,  0.1706,  0.1381,  0.1293,  0.0070]]],


        [[[-0.0370,  0.1529,  0.0437,  0.0769, -0.0402],
          [-0.1830, -0.1313,  0.1884, -0.0552,  0.0977],
          [-0.1966,  0.0685, -0.1407, -0.1272, -0.0549],
          [-0.0218,  0.1325, -0.1874,  0.0541, -0.1716],
          [-0.1121, -0.0900, -0.1455, -0.0328, -0.0427]]],


        [[[-0.1476,  0.0029, -0.1518,  0.0579, -0.0667],
          [ 0.0224,  0.1465,  0.1510, -0.1387, -0.0958],
          [ 0.0939, -0.0736,  0.1566, -0.1097,  0.0024],
          [-0.1572, -0.1649, -0.1314,  0.1645, -0.0210],
          [-0.0298, -0.1167, -0.1056,  0.1011, -0.1954]]],


        [[[ 0.0453,  0.0064, -0.1390,  0.1950,  0.1454],
          [-0.1826, -0.1646,  0.0784, -0.1364, -0.1568

In [42]:
def seed_everything(seed):
    torch.manual_seed(seed)       # Current CPU
    torch.cuda.manual_seed(seed)  # Current GPU
    np.random.seed(seed)          # Numpy module
    random.seed(seed)             # Python random module
    torch.backends.cudnn.benchmark = False    # Close optimization
    torch.backends.cudnn.deterministic = True # Close optimization
    torch.cuda.manual_seed_all(seed) # All GPU (Optional)
seed_everything(0)

In [114]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)


model = ResNet(ResidualBlock, [2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

Files already downloaded and verified
Epoch [1/80], Step [100/500] Loss: 1.6119


KeyboardInterrupt: 